# Housing Data in Israel

## Analysis across Israeli Cities and Districts for the Last 4 Years

### EDA Process:
- **Discover Null Values**: Identifying missing or incomplete data.
- **Separate out Key Data from Columns with Multiple Data Points in One Cell**: Extracting and structuring data that contains multiple pieces of information in a single cell.
- **Remove Null and "Total" Values from the Area Column**: Cleaning up the `"Area"` column by eliminating rows with missing or irrelevant values such as `"None"` and `"Total"`.
- **Convert Data Types to Appropriate Format**: Ensuring that columns (e.g., "Year", "Area") are in the correct data types (e.g., integer, string) for better analysis.
- **Impute missing housing data**



In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/teitelbaumsair/Desktop/Open-Source-Data-Projects/Housing_Data_Israel/Housing Data.csv')
df

,Code,Area and rooms of apartment,Currency,Year,Average,January-March,April-June,July-September,October-December
0,51000.0,Total,NIS thousand,2020.0,"1,600.6","1,621.4","1,542.5","1,571.7","1,629.2"
1,51010.0,1-2 (Total),NIS thousand,2020.0,"1,077.3","1,130.8",991.7,"1,052.3","1,100.9"
2,51030.0,3-2.5 (Total),NIS thousand,2020.0,"1,221.3","1,260.2","1,183.2","1,192.9","1,230.3"
3,51050.0,4-3.5 (Total),NIS thousand,2020.0,"1,582.5","1,604.5","1,540.8","1,553.8","1,599.7"
4,51070.0,5-4.5 (Total),NIS thousand,2020.0,"1,960.7","1,968.1","1,847.2","1,933.7","2,025.6"
...,...,...,...,...,...,...,...,...,...
710,53230.0,3-2.5 (Tel Aviv - 5000),NIS thousand,2024.0,-,"3,657.0","3,859.3","3,635.1",-
711,53250.0,4-3.5 (Tel Aviv - 5000),NIS thousand,2024.0,-,"4,967.3","4,824.5","4,866.1",-
712,53270.0,5-4.5 (Tel Aviv - 5000),NIS thousand,2024.0,-,"6,236.6","6,254.8","6,517.7",-
713,53290.0,6-5.5 (Tel Aviv - 5000),NIS thousand,2024.0,-,-,-,-,-


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Code                         714 non-null    float64
 1   Area and rooms of apartment  714 non-null    object 
 2   Currency                     714 non-null    object 
 3   Year                         714 non-null    float64
 4   Average                      714 non-null    object 
 5   January-March                714 non-null    object 
 6   April-June                   714 non-null    object 
 7   July-September               714 non-null    object 
 8   October-December             714 non-null    object 
dtypes: float64(2), object(7)
memory usage: 50.4+ KB


### Observations to focus on with this data:
- **value types**: Convert value types in  "Area and rooms of apartment", "Currency", "Average", "January-March", "April-June", "July-September", "October-December" from object to float.
- **bucket creation**: Area and rooms of apartment has multiple elements that need to be separated out into columns via one-hot encoding. They are: rooms of apartment, and City Name.
- **troublesome rows** There are rows that need to be removed as they contain totals for each year of the whole country combined that are not relevant to the granular data. They can be stored in a separate CSV for further analysis.
- **Code** can be dropped

In [4]:
df = df.drop(index=714)


In [5]:
df.replace('-', np.nan, inplace=True)

In [6]:
df.isna().sum()

Code                             0
Area and rooms of apartment      0
Currency                         0
Year                             0
Average                        151
January-March                   23
April-June                      24
July-September                  25
October-December               172
dtype: int64

In [7]:
# Function to split the column and extract needed data
def extract_room_and_city(value):
    # Check if the format matches the expected one (e.g., contains ' (')
    if '(' in value:
        try:
            # Split the room and city part
            room_part, area_part = value.split(' (')
            
            # Reverse the room numbers
            rooms = '-'.join(room_part.split('-')[::-1])
            
            # Extract city name and remove the number (5000)
            area = area_part.split(' -')[0]
            
            return rooms, area
        except Exception as e:
            return None, None  # Handle unexpected formatting gracefully
    else:
        return None, None  # In case the data doesn't follow the expected pattern

# Apply the function to the 'Area and rooms of apartment' column and create new columns
df[['Rooms', 'Area']] = df['Area and rooms of apartment'].apply(lambda x: pd.Series(extract_room_and_city(x)))

# Reorder columns to put 'Rooms' and 'Area' in index 1 and 2
# Assuming you want 'Rooms' at index 1 and 'Area' at index 2
cols = ['Area', 'Rooms'] + [col for col in df.columns if col not in ['Area', 'Rooms']]

# Reorder the DataFrame
df = df[cols]

# Optionally, drop the original 'Area and rooms of apartment' column
df.drop(columns=['Area and rooms of apartment'], inplace=True)

# Save the updated dataset
df.to_csv('Housing_Data_updated.csv', index=False)

In [8]:
df['Year'] = df['Year'].astype('Int64')

In [9]:
#drop rows where column area value is "None" or "Total)"
df = df[~df['Area'].isin(['Total)'])]
df = df.dropna(subset=['Area'])
df.head(10)

,Area,Rooms,Code,Currency,Year,Average,January-March,April-June,July-September,October-December
7,Jerusalem District,2-1,51110.0,NIS thousand,2020,"1,392.3","1,311.0","1,282.9","1,431.6","1,508.4"
8,Jerusalem District,2.5-3,51130.0,NIS thousand,2020,"1,460.0","1,541.7","1,478.6","1,443.4","1,402.3"
9,Jerusalem District,3.5-4,51150.0,NIS thousand,2020,"1,978.8","2,028.5","1,896.4","1,936.4","2,022.0"
10,Jerusalem District,4.5-5,51170.0,NIS thousand,2020,"2,296.3","2,232.2","2,218.6","2,257.7","2,421.6"
11,Jerusalem District,5.5-6,51190.0,NIS thousand,2020,"2,786.8","3,006.7","2,978.4","2,483.0","2,818.2"
13,North District,2-1,51210.0,NIS thousand,2020,432.4,418.5,425.5,440.3,449.4
14,North District,2.5-3,51230.0,NIS thousand,2020,653.9,626.1,645.8,655.9,681.2
15,North District,3.5-4,51250.0,NIS thousand,2020,967.0,933.8,956.1,950.2,"1,007.9"
16,North District,4.5-5,51270.0,NIS thousand,2020,"1,272.2","1,264.1","1,233.1","1,225.5","1,342.0"
17,North District,5.5-6,51290.0,NIS thousand,2020,"1,735.3","1,770.8","1,685.5","1,699.6","1,715.1"


In [10]:
df.isnull().sum()

Area                  0
Rooms                 0
Code                  0
Currency              0
Year                  0
Average             121
January-March        23
April-June           24
July-September       25
October-December    142
dtype: int64

In [11]:
#Next step is to fill the null values of each row by imputation 